(tfq)=

# Cirq & TFQ

## Введение

[Cirq](https://quantumai.google/cirq) это библиотека для работы с квантовыми компьютерами и симуляторами компании _Google_. В рамках темы квантового машинного обучения нам также интересен фреймворк [Tensorflow Quantum](https://www.tensorflow.org/quantum/overview). Это высокоуровневая библиотека, которая содержит готовые рутины для квантового и гибридного машинного обучения. В качестве системы автоматического дифференцирования, а также для построения гибридных квантово-классических нейронных сетей там используется библиотека [Tensorflow](https://www.tensorflow.org/overview/).

```{warning}
Во всех дальнейших лекциях мы будем использовать в основном библиотеку `PennyLane`, так что данная лекция исключительно обзорная и факультативная. В ней мы посмотрим несколько примеров _end2end_ обучения квантовых схем на `TFQ` без детального объяснения теории и вообще того, что происходит. Основная цель данной лекции это исключительно обзор еще одного инструмента, а не изучение QML! Заинтересованный читатель может вернуться к этому обзору после изучения глав про **VQC**, **Градиенты** и **Квантовые нейросети**.
```

## Работа с кубитами

### Импорты и схема

Для начала импорты.

In [1]:
import cirq

`Cirq` рассчитан на работу с квантовым компьютером от компании _Google_, который представляет из себя решетку кубитов. Поэтому в нем все строится вокруг работы с решеткой кубитов -- объектом `cirq.GridQubit`. Давайте создадим кубит на решетке, который имеет координаты $(0, 0)$:

In [2]:
qubit = cirq.GridQubit(0, 0)

Следующей важной концепцией в `Cirq` является непосредственно квантовая схема. Давайте создадим схему, которая переводит кубит в суперпозицию состояний $\ket{0}$ и $\ket{1}$ и измеряет его:

In [3]:
circuit = cirq.Circuit()
circuit.append(cirq.H(qubit))
circuit.append(cirq.measure(qubit))
print(circuit)

(0, 0): ───H───M───


### Запуск и симуляция

Теперь создадим квантовый симулятор, который посчитает нам результат этой простой схемы на классическом компьютере:

In [4]:
sim = cirq.Simulator()

Как мы знаем, результат измерения такой схемы равен 50% для состояния $\ket{0}$, то есть если мы будем сэмплировать, то должны получать $\sim 0.5$. Проверим это с разным числом сэмплов:

In [5]:
print("5 сэмплов:")
print(sim.sample(circuit, repetitions=5).mean())
print("\n100 сэмплов:")
print(sim.sample(circuit, repetitions=100).mean())
print("\n500 сэмплов:")
print(sim.sample(circuit, repetitions=500).mean())

5 сэмплов:
(0, 0)    0.6
dtype: float64

100 сэмплов:
(0, 0)    0.47
dtype: float64

500 сэмплов:
(0, 0)    0.48
dtype: float64


```{note}
Метод `sim.sample` озвращает хорошо знакомый всем специалистам в области Data Science объект `pandas.DataFrame`. Для тех, кто слышит про такой впервые рекомендуем обратиться к вводным лекциям про `Python` и классическое машинное обучение.
```

Также у нас есть опция запустить схему через метод `run`. Может показаться, что это то же самое, но на самом деле в отличии от `sample` метод `run` возвращает результат в несколько ином виде, а еще он позволяет запускать программу на реальном квантовом компьютере `Goolge` или их новых квантовых симуляторах на TPU:

In [6]:
print(sim.run(circuit, repetitions=25))

(0, 0)=1100001101010111000110000


Тут мы просто видим последовательность наших измерений.

## Квантовое машинное обучение

### Импорты

Мы будем использовать `Tensorflow` и `Tensorflow Quantum`.

In [7]:
import tensorflow as tf
import tensorflow_quantum as tfq

`Tensorflow Quantum` позволяет "превращать" параметризированные схемы `Cirq` в слои нейронных сетей `Tensorflow`. Но для начала нам все равно потребуется схема. Давайте объявим парку кубит, для этого воспользуемся другим типом кубитов -- цепочкой:

In [8]:
qubits = cirq.LineQubit.range(2)
print(qubits)

[cirq.LineQubit(0), cirq.LineQubit(1)]


Давайте попробуем решить игрушечную задачку классификации простой гибридной квантово-классической нейронной сетью. У нас будет один квантовый слой и один классический слой. В качестве задачи возьмем набор данных `Two Moons`, к которому мы также будем часто образаться в других лнециях.

In [9]:
from sklearn.datasets import make_moons
x, y = make_moons(25)